# Prévision de la mortalité (France, 2000–2022)

## Ce projet étudie la mortalité de la population française sur la période 2000–2022.

## I - Construction des données:

 Les données exploitées dans cette étude proviennent de la Human Mortality Database
 (https://www.mortality.org), plateforme spécialisée dans la collecte et la diffusion de
 statistiques démographiques longitudinales. Après inscription, nous avons pu accéder à deux
 fichiers essentiels concernant la population française.

### 1 -  Prétraitement de la base des décès (Death counts):

La première base téléchargée concerne le **nombre annuel de décès** selon trois dimensions :

- **Year** : année civile *(t)*
- **Age** : âge exact *(x)*, variant de **0** à **100** ans
- **Sexe** : *(i = Female, Male)*, et **Total**

Après copie manuelle depuis la plateforme vers un **fichier texte** via *Notepad*, les données ont été importées dans **Excel**.  
Nous avons structuré la base selon les variables suivantes :

- **Year** : année civile *(t)*
- **Age** : âge exact *(x)*
- **D_F** : nombre de décès **féminins**
- **D_H** : nombre de décès **masculins**
- **D_Tot** : nombre de décès **totaux**

Cette base couvre initialement la période **1860–2022**, mais nous avons restreint l’échantillon à la période **2000–2022** pour des raisons de qualité, de modernité statistique et de pertinence actuarielle.


### 2 -  Prétraitement de la base des expositions (Exposure-to-risk)

La deuxième base contient les **expositions au risque de décès**, c’est-à-dire le nombre moyen d’individus vivants à chaque âge et pour chaque année.  
Les variables sont structurées de manière identique :

- **Year** : année civile *(t)*
- **Age** : âge exact *(x)*
- **E_F** : exposition **féminine**
- **E_H** : exposition **masculine**
- **E_Tot** : exposition **totale** (somme des deux sexes)

Les données ont été importées dans un second fichier Excel à partir d’un fichier texte brut, suivant le même protocole que pour les décès.  
La période analysée est également restreinte à **2000–2022**.


### 3 - Fusion des deux bases

Les deux bases ont été fusionnées par **jointure interne** sur les clés communes :

**Year = t**    et    **Age = x**

Le jeu de données final comporte donc les colonnes suivantes :

- **Year**, **Age**
- **E_F**, **E_H**, **E_Tot**
- **D_F**, **D_H**, **D_Tot**


### 4 - Construction des taux de mortalité

À partir des données fusionnées, nous construisons les **taux de mortalité centraux** :

- **Par sexe** :

m(t, x, i) = D(t, x, i)\E(t, x, i) avec i dans F, H

- **Agrégé** :

m(t, x) = D(t, x)\E(t, x)

Ces taux m_F, m_H et m_Tot constituent la base d’entrée pour notre étude.

In [13]:
import pandas as pd
import numpy as np
!pip install openpyxl


# 1. Chemin vers le fichier Excel (la "base jaune" fusionnée)
# 👉 adapte le chemin/nom selon ton projet
file_path = "Mortalite France .xlsx"

# 2. Importer les données
df = pd.read_excel(file_path)

# 3. Afficher l'entête pour vérifier
print("Colonnes disponibles dans le fichier :")
print(df.columns.tolist())
print("\nAperçu des premières lignes :")
print(df.head())

# 4. Harmoniser les noms de colonnes (si besoin)
#    On gère les cas 'Année' / 'Annee', etc.
df = df.rename(columns={
    "Année": "Year",
    "Annee": "Year",
    "annee": "Year",
    "AGE": "Age",
    "age": "Age"
})

# 5. Sélectionner et ordonner les colonnes utiles
colonnes_attendues = ["Year", "Age", "E_F", "E_H", "E_Tot", "D_F", "D_H", "D_Tot"]

# Vérification des colonnes manquantes
missing = [c for c in colonnes_attendues if c not in df.columns]
if missing:
    print("\n⚠️ Attention : colonnes manquantes dans le fichier :", missing)

# On ne garde que les colonnes existantes parmi celles attendues
colonnes_finales = [c for c in colonnes_attendues if c in df.columns]
df = df[colonnes_finales].copy()

# 6. Calcul des taux de mortalité
# m_F(t,x) = D_F(t,x) / E_F(t,x)
# m_H(t,x) = D_H(t,x) / E_H(t,x)
# m_Tot(t,x) = D_Tot(t,x) / E_Tot(t,x)
# On évite la division par zéro avec np.where

if {"D_F", "E_F"}.issubset(df.columns):
    df["m_F"] = np.where(df["E_F"] > 0, df["D_F"] / df["E_F"], np.nan)

if {"D_H", "E_H"}.issubset(df.columns):
    df["m_H"] = np.where(df["E_H"] > 0, df["D_H"] / df["E_H"], np.nan)

if {"D_Tot", "E_Tot"}.issubset(df.columns):
    df["m_Tot"] = np.where(df["E_Tot"] > 0, df["D_Tot"] / df["E_Tot"], np.nan)

# 7. Trier par année et âge
if {"Year", "Age"}.issubset(df.columns):
    df = df.sort_values(["Year", "Age"]).reset_index(drop=True)

# 8. Afficher l'entête de la base finale
pd.set_option("display.max_columns", None)
print("\nAperçu de la base finale avec les taux :")
print(df.head())
print("\nColonnes :", df.columns.tolist())

# 9. Sauvegarder la base propre (optionnel mais recommandé)
output_excel = "mortalite_france_2000_2022_processed.xlsx"
output_csv = "mortalite_france_2000_2022_processed.csv"

df.to_excel(output_excel, index=False)
df.to_csv(output_csv, index=False)

print(f"\n✅ Fichiers enregistrés :\n- {output_excel}\n- {output_csv}")


Colonnes disponibles dans le fichier :
['Annee', 'Age', 'E_F', 'E_H', 'E_Tot', 'D_F', 'D_H', 'D_Tot']

Aperçu des premières lignes :
   Annee  Age        E_F        E_H      E_Tot   D_F   D_H  D_Tot
0   2000    0  362246.66  380982.78  743229.43  1425  1992   3417
1   2000    1  354288.33  371892.85  726181.18   149   158    307
2   2000    2  350344.21  368268.07  718612.28    69    96    165
3   2000    3  351312.27  370227.17  721539.45    57    82    139
4   2000    4  353345.34  371065.57  724410.91    46    65    111

Aperçu de la base finale avec les taux :
   Year  Age        E_F        E_H      E_Tot   D_F   D_H  D_Tot       m_F  \
0  2000    0  362246.66  380982.78  743229.43  1425  1992   3417  0.003934   
1  2000    1  354288.33  371892.85  726181.18   149   158    307  0.000421   
2  2000    2  350344.21  368268.07  718612.28    69    96    165  0.000197   
3  2000    3  351312.27  370227.17  721539.45    57    82    139  0.000162   
4  2000    4  353345.34  371065.57  724

## II - Validation des données

### 1 - Qualité des données

Il est important de souligner que les fichiers extraits de la *Human Mortality Database* sont déjà :

- **nettoyés** : pas de doublons, pas de formats incohérents
- **cohérents** : les valeurs d’exposition et de décès sont compatibles et correctement agrégées
- **fiables** : les estimations sont produites selon une méthodologie rigoureuse conforme aux normes démographiques internationales

Aucune anomalie n’a été détectée lors de l’importation ni dans le traitement des variables,
ce qui rend la base directement exploitable pour des travaux de modélisation.


### 2 - Validation finale de la base de données

In [14]:
import pandas as pd
import numpy as np

# 1. Charger la base finale (feuille "Mortality")
# 👉 adapte le chemin et le nom de fichier si besoin
file_path = "mortalite_france_2000_2022_processed.xlsx"

print("Aperçu des premières lignes :")
print(df.head())

print("\nColonnes disponibles :")
print(df.columns.tolist())

# 2. Harmoniser les noms si nécessaire
df = df.rename(columns={
    "Année": "Annee",
    "Année ": "Annee",
    "Year": "Annee",
    "age": "Age",
    "E_H": "E_M"   # si tu utilises E_H dans le fichier, on le renomme en E_M pour être cohérent avec le texte
})

# 3. Vérifier la période temporelle et la plage d’âges
annee_min, annee_max = df["Annee"].min(), df["Annee"].max()
age_min, age_max = df["Age"].min(), df["Age"].max()

print(f"\nPériode couverte : {annee_min} – {annee_max}")
print(f"Plage d'âges couverte : {age_min} – {age_max}")

# 4. Vérifier unicité (Annee, Age)
nb_lignes = len(df)
nb_uniques = df[["Annee", "Age"]].drop_duplicates().shape[0]
print(f"\nNombre total de lignes : {nb_lignes}")
print(f"Nombre de couples (Annee, Age) uniques : {nb_uniques}")

if nb_lignes == nb_uniques:
    print("✅ Chaque ligne correspond à une combinaison unique (Annee, Age).")
else:
    print("⚠️ Des doublons existent sur (Annee, Age). À vérifier.")

# 5. Vérifier égalités d’agrégation
checks = {}

if {"E_F", "E_M", "E_Tot"}.issubset(df.columns):
    diff_E = (df["E_Tot"] - (df["E_F"] + df["E_M"])).abs().max()
    checks["E_Tot = E_F + E_M (max écart)"] = diff_E

if {"D_F", "D_H", "D_Tot"}.issubset(df.columns):
    diff_D = (df["D_Tot"] - (df["D_F"] + df["D_H"])).abs().max()
    checks["D_Tot = D_F + D_H (max écart)"] = diff_D

print("\nVérification des égalités d'agrégation :")
for k, v in checks.items():
    print(f"- {k} : {v}")

# 6. Vérifier absence de valeurs manquantes
na_counts = df.isna().sum()
print("\nNombre de valeurs manquantes par colonne :")
print(na_counts)

# 7. Vérifier la cohérence des types et la positivité
print("\nTypes des colonnes :")
print(df.dtypes)

# Variables censées être entières
int_vars = ["Annee", "Age", "D_F", "D_H", "D_Tot"]
for col in int_vars:
    if col in df.columns:
        non_int = ((df[col] % 1) != 0).sum()
        print(f"\n{col} : {non_int} valeurs non entières détectées.")

# Variables censées être positives
pos_vars = ["E_F", "E_M", "E_Tot", "m_F", "m_H", "m_Tot"]
for col in pos_vars:
    if col in df.columns:
        min_val = df[col].min()
        print(f"{col} : min = {min_val}")

# 8. Petit résumé automatique
print("\n📌 Résumé :")
if (annee_min == 2000) and (annee_max == 2022):
    print("- Période conforme : 2000–2022.")
else:
    print("- ⚠️ Période différente de 2000–2022, à vérifier.")

if (age_min == 0) and (age_max == 100):
    print("- Plage d’âges conforme : 0–100 ans.")
else:
    print("- ⚠️ Plage d’âges différente de 0–100 ans, à vérifier.")

if df.isna().sum().sum() == 0:
    print("- Aucune valeur manquante détectée.")
else:
    print("- ⚠️ Des valeurs manquantes existent, à traiter.")

if checks:
    ok_agg = all(v == 0 for v in checks.values())
    if ok_agg:
        print("- Égalités d’agrégation (E_Tot, D_Tot) respectées.")
    else:
        print("- ⚠️ Écarts dans les égalités d’agrégation, à examiner.")

print("- La base est prête pour la modélisation (si tous les voyants sont au vert ci-dessus).")


Aperçu des premières lignes :
   Year  Age        E_F        E_H      E_Tot   D_F   D_H  D_Tot       m_F  \
0  2000    0  362246.66  380982.78  743229.43  1425  1992   3417  0.003934   
1  2000    1  354288.33  371892.85  726181.18   149   158    307  0.000421   
2  2000    2  350344.21  368268.07  718612.28    69    96    165  0.000197   
3  2000    3  351312.27  370227.17  721539.45    57    82    139  0.000162   
4  2000    4  353345.34  371065.57  724410.91    46    65    111  0.000130   

        m_H     m_Tot  
0  0.005229  0.004598  
1  0.000425  0.000423  
2  0.000261  0.000230  
3  0.000221  0.000193  
4  0.000175  0.000153  

Colonnes disponibles :
['Year', 'Age', 'E_F', 'E_H', 'E_Tot', 'D_F', 'D_H', 'D_Tot', 'm_F', 'm_H', 'm_Tot']

Période couverte : 2000 – 2022
Plage d'âges couverte : 0 – 100

Nombre total de lignes : 2323
Nombre de couples (Annee, Age) uniques : 2323
✅ Chaque ligne correspond à une combinaison unique (Annee, Age).

Vérification des égalités d'agrégation :


Après l’application des traitements décrits précédemment, notamment le contrôle des ratios d’exposition et de décès, une **vérification complète de la cohérence des variables** a été effectuée.

Pour chaque variable de la base consolidée (*Year, Age, E_F, E_H, E_Tot, D_F, D_H, D_Tot, m_F, m_H, m_Tot*), nous avons vérifié que :

- les valeurs sont strictement positives ou nulles, sans valeurs aberrantes,
- les taux de mortalité restent logiques, compris dans l’intervalle \([0,1]\), et croissent de manière réaliste avec l’âge,
- les égalités d’agrégation sont respectées :

Aucune anomalie résiduelle n’a été détectée à ce stade.  
La base de données peut donc être considérée comme **propre, cohérente et exploitable** pour l’étude des dynamiques de mortalité, que ce soit via des approches actuarielles classiques ou des méthodes d’apprentissage automatique.


### 3 - Description technique de la base

La base de données finale  à utiliser concerne la population **civile française**, sur la période **2000 à 2022**, et résulte de la fusion des données d’exposition et de décès, accompagnée du calcul explicite des taux de mortalité.

Elle est structurée sous forme tabulaire de **2323 lignes** , chaque ligne correspondant à une combinaison unique d’année (**Year**) et d’âge (**Age**). Elle couvre les âges allant de **0 à 100 ans** pour chaque année entre **2000 et 2022**.

**Les variables présentes sont les suivantes :**

- **Year** : année civile (entier, de 2000 à 2022)
- **Age** : âge exact (de 0 à 100)
- **E_F** : exposition féminine \(E(t, x, F)\)
- **E_M** : exposition masculine \(E(t, x, H)\)
- **E_Tot** : exposition totale (somme des deux sexes) \(E(t, x) = E_F + E_M\)
- **D_F** : décès féminins \(D(t, x, F)\)
- **D_H** : décès masculins \(D(t, x, H)\)
- **D_Tot** : décès totaux (hommes + femmes) \(D(t, x) = D_F + D_H\)
- **m_F** : taux de mortalité féminin
- **m_H** : taux de mortalité masculin
- **m_Tot** : taux de mortalité total

Toutes les variables sont renseignées **sans valeurs manquantes**, et les types de données sont cohérents :
- **Year**, **Age**, **D_F**, **D_H**, **D_Tot** sont des entiers.
- **E_F**, **E_M**, **E_Tot**, **m_F**, **m_H**, **m_Tot** sont des réels positifs.

Cette base est désormais prête à être utilisée pour les étapes suivantes de l’étude, notamment l’entraînement et la validation des modèles de prévision de la mortalité.
